<a href="https://colab.research.google.com/github/M-Jak/Finki/blob/main/Voved%20vo%20nauka%20na%20podatoci/AUD%20scraping/custom_setec_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
from IPython.display import HTML

import warnings

requests.packages.urllib3.disable_warnings()
warnings.filterwarnings("ignore")

from google.colab import drive

drive.mount('/content/drive')

In [79]:
response = requests.get("https://setec.mk/index.php?route=product/category&path=10019_10020&sort=p.sort_order&order=ASC")

In [80]:
html = BeautifulSoup(response.text,"html.parser")
# name -> div.right -> a -> text -> strip
# price -> div.right -> span.price-old-new -> text
# price ackiska -> div.right -> span.price-new-new -> text
# shifra -> div.right -> div.shifra -> text
# html.select_one('div.right').select_one("a").text.strip()

In [87]:
def scrape(url="https://setec.mk/index.php?route=product/category&path=10019_10020&sort=p.sort_order&order=ASC&limit=100",\
           path="/content/drive/MyDrive/VNP labs/AUD scraping/data"):
  response = requests.get(url)
  html = BeautifulSoup(response.text,"html.parser")
  print("Response status code: " + str(response.status_code))
  if response.status_code != 200:
    return None
  produkti_html = html.find_all("div",{"class":"right"})
  products = []

  for i in range(len(produkti_html)):
    item = {}
    item['ID'] = i+1
    item['Name'] = produkti_html[i].select_one('a').text.strip()
    item['Regular price'] = produkti_html[i].select_one('span.price-old-new').text
    item['Discounted price'] = produkti_html[i].select_one('span.price-new-new').text
    item['Shifra'] = produkti_html[i].select_one('div.shifra').text.split(":")[1].strip()

    products.append(item)

  df = pd.DataFrame(products)
  url_parts = url.split("/")
  filename = path + "/results_" + url_parts[2].split(".")[0] + ".csv"
  df.to_csv(filename, index=False)
  return df

In [ ]:
data = scrape("https://setec.mk/index.php?route=product/category&path=10066_10067&sort=p.price&order=DESC")
if data is None:
  print("Response was not OK, cannot scrape.")